In [2]:
import pandas as pd

In [3]:
csvpath = 'Resources/purchase_data.csv'
purchases_df = pd.read_csv(csvpath)


In [69]:
# Total Players
total_players = len(purchases_df['SN'].unique())
print(f'Total number of House of Pymoli players: {total_players}')

Total number of House of Pymoli players: 576


In [70]:
# Purchasing Analysis

## Total Items
total_items = len(purchases_df['Item ID'].unique())


# Average Purchase Price
price_sum = purchases_df['Price'].sum()
average_purchase_price = price_sum/total_items


# Total Purchases
total_purchases = len(purchases_df)



summary_df = pd.DataFrame({'Number of Unique Items':[total_items],'Average Purchase Price':[average_purchase_price],
                          'Total Purchases':[total_purchases],'Total Revenue':[price_sum]})
summary_df['Average Purchase Price'] = summary_df['Average Purchase Price'].map('${:,.2f}'.format)
summary_df['Total Revenue'] = summary_df['Total Revenue'].map('${:,.2f}'.format)
summary_df

,Number of Unique Items,Average Purchase Price,Total Purchases,Total Revenue
0,179,$13.29,780,"$2,379.77"


In [71]:
## Gender Demographics

## Create new data representing Users, Gender and Age
user_info_df = purchases_df[['SN','Gender','Age']]
clean_user_df = user_info_df.drop_duplicates(keep='first')

# Find out unique values in gender column
genders = user_info_df['Gender'].unique()

## Percentage and Count of Female Players
female_players = clean_user_df.loc[clean_user_df['Gender']=='Female']
total_female = len(female_players)
percent_female = round((total_female/total_players)*100,1)

## Percentage and Count of Male Players
male_players = clean_user_df.loc[clean_user_df['Gender']=='Male']
total_male = len(male_players)
percent_male = round((total_male/total_players)*100,1)

## Percentage and Count of Other / Non-Disclosed
other_players = clean_user_df.loc[clean_user_df['Gender']=='Other / Non-Disclosed']
total_other = len(other_players)
percent_other = round((total_other/total_players)*100,1)


gender_summary_df = pd.DataFrame({'Gender':genders,'Total Players':[total_male,total_other,total_female],
                                  'Percent of Total Players':[percent_male,percent_other,percent_female]})

gender_summary_df['Percent of Total Players'] = gender_summary_df['Percent of Total Players'].map('{:,.1f}%'.format)
gender_summary_df

,Gender,Total Players,Percent of Total Players
0,Male,484,84.0%
1,Other / Non-Disclosed,11,1.9%
2,Female,81,14.1%


In [72]:
### Purchasing Analysis (Gender) - Purchase Count

## Female total purchases
female_df = purchases_df.loc[purchases_df['Gender']=='Female']
female_count = female_df['Item ID'].count()


## Male total purchases
male_df = purchases_df.loc[purchases_df['Gender']=='Male']
male_count = male_df['Item ID'].count()
x = male_df['Price'].sum()

## Other/Non-Disclosed total purchases
other_df = purchases_df.loc[purchases_df['Gender']=='Other / Non-Disclosed']
other_count = other_df['Item ID'].count()


gender_group_df = purchases_df.groupby(['Gender']).sum()

## Female 
female_total_price = round(gender_group_df.iloc[0,3],2)
female_ave_price = round(female_total_price/female_count,2)
price_per_female = round(female_total_price/total_female,2)


## Male
male_total_price = round(gender_group_df.iloc[1,3],2)
male_ave_price = round(male_total_price/male_count,2)
price_per_male = round(male_total_price/total_male,2)


## Other/Non-disclosed
other_total_price = round(gender_group_df.iloc[2,3],2)
other_ave_price = round(other_total_price/other_count,2)
price_per_other = round(other_total_price/total_other,2)









gender_purchase_df = pd.DataFrame({'Gender':genders,'Total Purchased':[male_count,other_count,female_count],
                                  'Average Purchase Price':[male_ave_price,other_ave_price,female_ave_price],
                                  'Total Purchase Value':[male_total_price,other_total_price,female_total_price],
                                  'Average Price Per Gender':[price_per_male,price_per_other,price_per_female]})

gender_purchase_df['Average Purchase Price'] = gender_purchase_df['Average Purchase Price'].map('${:,.2f}'.format)
gender_purchase_df['Total Purchase Value'] = gender_purchase_df['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_df['Average Price Per Gender'] = gender_purchase_df['Average Price Per Gender'].map('${:,.2f}'.format)


gender_purchase_df

,Gender,Total Purchased,Average Purchase Price,Total Purchase Value,Average Price Per Gender
0,Male,652,$3.02,"$1,967.64",$4.07
1,Other / Non-Disclosed,15,$3.35,$50.19,$4.56
2,Female,113,$3.20,$361.94,$4.47


In [73]:
## Age Demographics - Create Data Frame to seperate users by age into bins

## Create new Data Frames for Age Analysis
## unique_age_df
age_demo_df = purchases_df[['SN','Age','Purchase ID','Item Name','Price']]
unique_age_df = clean_user_df.copy()


## create bins to sort users into age groups 
bins = [0, 20, 30, 40, 100]
age_groups = ['20 and Under', '21 to 30', '31 to 40','41 and Older']

## Group by Data Frame to get count, mean and sum by age group
age_demo_df['Age Group'] = pd.cut(age_demo_df['Age'], bins, labels = age_groups, include_lowest=True)
age_group_df = age_demo_df.groupby(['Age Group']).agg({'Purchase ID':'count','Price':['mean','sum']})
age_group_df.columns = age_group_df.columns.map('_'.join)
age_group_df = age_group_df.rename(columns={'Purchase ID_count':'Total Purchases','Price_mean':'Average Price',
                                           'Price_sum':'Total Sales'})
age_group_df = age_group_df.reset_index(drop=False)


unique_age_df['Age Group'] = pd.cut(unique_age_df['Age'], bins, labels = age_groups, include_lowest=True)
no_dup_age_df = unique_age_df.groupby(['Age Group']).count()

## total users in each age group using clean_user_df containing unique users
uni_teens = no_dup_age_df.iloc[0,0]
uni_yg_adults = no_dup_age_df.iloc[1,0]
uni_adults = no_dup_age_df.iloc[2,0]
uni_mid_adults = no_dup_age_df.iloc[3,0]

teen = age_group_df.iloc[0,3]/uni_teens
yg_adult = age_group_df.iloc[1,3]/uni_yg_adults
adult = age_group_df.iloc[2,3]/uni_adults
wise_adult = age_group_df.iloc[3,3]/uni_mid_adults


age_group_df['Average Price Per Age'] = [teen,yg_adult,adult,wise_adult]

age_group_df['Average Price'] = age_group_df['Average Price'].map('${:,.2f}'.format)
age_group_df['Total Sales'] = age_group_df['Total Sales'].map('${:,.2f}'.format)
age_group_df['Average Price Per Age'] = age_group_df['Average Price Per Age'].map('${:,.2f}'.format)


age_group_df







<ipython-input-73-06563a569f4e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_demo_df['Age Group'] = pd.cut(age_demo_df['Age'], bins, labels = age_groups, include_lowest=True)


,Age Group,Total Purchases,Average Price,Total Sales,Average Price Per Age
0,20 and Under,286,$3.10,$887.12,$4.13
1,21 to 30,402,$2.99,"$1,203.06",$4.13
2,31 to 40,85,$3.15,$268.06,$4.25
3,41 and Older,7,$3.08,$21.53,$3.08


In [74]:
## Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

new_group = age_demo_df.groupby(['SN']).agg({'Purchase ID':['count'],
                                                    'Price':['sum']})
new_group.columns = new_group.columns.map('_'.join)

sorted_group = new_group.sort_values(['Price_sum'],ascending=False).head()

mylist = []
for i in range(0,5):
    price_sum = sorted_group['Price_sum'][i]
    purchase_count = sorted_group['Purchase ID_count'][i]
    average_price2 = round(price_sum/purchase_count,2)
    mylist.append(average_price2)


sorted_group['Average purchase Price'] = mylist
sorted_group = sorted_group.rename(columns={'Purchase ID_count':'Purchases',
                                           'Price_sum':'Total Price',
                                            'Average purchase Price':'Average Price'})

sorted_group['Total Price'] = sorted_group['Total Price'].map('${:,.2f}'.format)
sorted_group['Average Price'] = sorted_group['Average Price'].map('${:,.2f}'.format)
sorted_group = sorted_group.reset_index(drop=False)

sorted_group


,SN,Purchases,Total Price,Average Price
0,Lisosia93,5,$18.96,$3.79
1,Idastidru52,4,$15.45,$3.86
2,Chamjask73,3,$13.83,$4.61
3,Iral74,4,$13.62,$3.40
4,Iskadarya95,3,$13.10,$4.37


In [75]:
## Identify the 5 most popular items by purchase count, then list (in a table):

items_df = purchases_df[['Item Name','Item ID','Price','SN']]
group_items_df = items_df.groupby(['Item Name','Item ID']).agg({'Price':['max','sum'],
                                                                'SN':'count'})
group_items_df.columns = group_items_df.columns.map('_'.join)
group_items_df = group_items_df.rename(columns={'Price_max':'Price',
                                               'Price_sum':'Total Sales',
                                               'SN_count':'Total Sold'})

group_items_df = group_items_df.reset_index(drop=False)
sorted_items_df = group_items_df.sort_values(['Total Sold'],ascending=False)

final_items_df = sorted_items_df.reset_index(drop=True).head()
final_items_df['Price'] = final_items_df['Price'].map('${:,.2f}'.format)
final_items_df['Total Sales'] = final_items_df['Total Sales'].map('${:,.2f}'.format)



final_items_df


    


,Item Name,Item ID,Price,Total Sales,Total Sold
0,Final Critic,92,$4.88,$59.99,13
1,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,$50.76,12
2,Persuasion,132,$3.33,$28.99,9
3,Nirvana,82,$4.90,$44.10,9
4,"Extraction, Quickblade Of Trembling Hands",108,$3.53,$31.77,9


In [76]:
## Identify the 5 most profitable items by total purchase value, then list (in a table):

total_value_df = group_items_df.sort_values(['Total Sales'],ascending=False)
final_total_value_df = total_value_df.reset_index(drop=True).head()

final_total_value_df['Price'] = final_total_value_df['Price'].map('${:,.2f}'.format)
final_total_value_df['Total Sales'] = final_total_value_df['Total Sales'].map('${:,.2f}'.format)


final_total_value_df


,Item Name,Item ID,Price,Total Sales,Total Sold
0,Final Critic,92,$4.88,$59.99,13
1,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,$50.76,12
2,Nirvana,82,$4.90,$44.10,9
3,Fiery Glass Crusader,145,$4.58,$41.22,9
4,Singed Scalpel,103,$4.35,$34.80,8
